In [1]:
def get_filing(file):
    fileA = open("filing.txt",'r')
    fileX = open(file,'r')
    s = fileX.readlines()
    l = ""
    for ele in s:
        l +=ele.replace("\n","")
    lst = l.split()
    # print(lst)
    s = fileA.readlines()
    l = ""
    for ele in s:
        l +=(ele.replace("\n",""))+" "
    fr = l.split(".")
    ans = []
    for ele in fr:
        ele = ele.strip()
        if any(word in ele for word in lst):
            ans.append(ele)
    fileA.close()
    fileX.close()
    return ans

In [2]:
def write(file,data):
    f = open(file,"w")
    for ele in data:
        f.write(ele+".\n")
    f.close()

In [3]:
l = ["theme.txt"]
for ele in l:
    ans = get_filing(ele)
    write("uni"+ele,ans)

In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
with open('unitheme.txt', 'r') as file:
    dataA = file.read().replace('\n', '')    
with open('theme.txt', 'r') as file:
    dataB = file.read().replace('\n', '')    

In [6]:
bagOfWordsA = dataA.split()
bagOfWordsB = dataB.split()
uniqueWords = set(bagOfWordsA).union(set(bagOfWordsB))
numOfWordsA = dict.fromkeys(uniqueWords, 0)
for word in bagOfWordsA:
    numOfWordsA[word] += 1
numOfWordsB = dict.fromkeys(uniqueWords, 0)
for word in bagOfWordsB:
    numOfWordsB[word] += 1

In [7]:
from nltk.corpus import stopwords
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [8]:
def computeTF(wordDict, bagOfWords):
    tfDict = {}
    bagOfWordsCount = len(bagOfWords)
    for word, count in wordDict.items():
        tfDict[word] = count / float(bagOfWordsCount)
    return tfDict    

In [9]:
tfA = computeTF(numOfWordsA, bagOfWordsA)
tfB = computeTF(numOfWordsB, bagOfWordsB)

In [10]:
def computeIDF(documents):
    import math
    N = len(documents)
    
    idfDict = dict.fromkeys(documents[0].keys(), 0)
    for document in documents:
        for word, val in document.items():
            if val > 0:
                idfDict[word] +=1
                
    for word, val in idfDict.items():
        idfDict[word] = math.log(N / float(val))
    return idfDict

In [11]:
idfs = computeIDF([numOfWordsA, numOfWordsB])

In [12]:
def computeTFIDF(tfBagOfWords, idfs):
    tfidf = {}
    for word, val in tfBagOfWords.items():
        tfidf[word] = val * idfs[word]
    return tfidf

In [13]:
tfidfA = computeTFIDF(tfA, idfs)
tfidfB = computeTFIDF(tfB, idfs)
df = pd.DataFrame([tfidfA, tfidfB])

In [14]:
listA = list(tfidfA.values())
listB = list(tfidfB.values())

In [15]:
from sklearn.metrics.pairwise import cosine_similarity
csim = cosine_similarity([listA],[listB])